In [1]:
from qibullet import SimulationManager
# from qibullet import RomeoVirtual
from qibullet import PepperVirtual
# from qibullet import NaoVirtual
import numpy as np
import time
import pybullet as p
import cv2
import math
from poseDetection import PoseDetection
from random import random
import keras
import csv
import pickle as pkl

In [2]:
def euler2quaternion(yaw, pitch, roll):
    qx = np.sin(roll/2) * np.cos(pitch/2) * np.cos(yaw/2) - np.cos(roll/2) * np.sin(pitch/2) * np.sin(yaw/2)
    qy = np.cos(roll/2) * np.sin(pitch/2) * np.cos(yaw/2) + np.sin(roll/2) * np.cos(pitch/2) * np.sin(yaw/2)
    qz = np.cos(roll/2) * np.cos(pitch/2) * np.sin(yaw/2) - np.sin(roll/2) * np.sin(pitch/2) * np.cos(yaw/2)
    qw = np.cos(roll/2) * np.cos(pitch/2) * np.cos(yaw/2) + np.sin(roll/2) * np.sin(pitch/2) * np.sin(yaw/2)

    return [qx, qy, qz, qw]


In [3]:
def clamp(num, min_value, max_value):
    return float(max(min(num, max_value), min_value))

In [4]:
simulation_manager = SimulationManager()
client_id = simulation_manager.launchSimulation(gui=True)

In [5]:
# Loading pepper
pepper = simulation_manager.spawnPepper(client_id,spawn_ground_plane=True)
pepper2 = simulation_manager.spawnPepper(client_id,translation=[1.5, 0, 0], quaternion = euler2quaternion(math.pi,0,0))
# nao = simulation_manager.spawnNao(client_id,translation=[0, 2, 0],quaternion=[0, 0, 0, 1])
# romeo = simulation_manager.spawnRomeo(client_id,translation=[0, 4, 0],spawn_ground_plane=False)

In [6]:
p.connect(p.DIRECT)

1

In [7]:
handle2 = pepper.subscribeCamera(PepperVirtual.ID_CAMERA_TOP)
handle3 = pepper2.subscribeCamera(PepperVirtual.ID_CAMERA_TOP)

In [8]:
pd = PoseDetection(model_complexity = 2)

In [10]:
# Controlling joints
joint_parameters = list()

for name, joint in pepper.joint_dict.items():
    if "Shoulder" in name or "Elbow" in name:
        joint_parameters.append((name,joint.getLowerLimit(),joint.getUpperLimit()))


In [ ]:
# Keras serialize model

landmarks = dict()
previous_time = time.time()
model = keras.models.load_model("model")
changed = False
try:
    while True:
        # Get pose from camera
        img = pepper.getCameraFrame(handle2)
        img2 = pepper2.getCameraFrame(handle3)
        img_og = img.copy()
        results = pd.getPose(img,draw_joints =True, draw_connections = True, only_arms_connections = True)
        if results.pose_landmarks:
            for id,lm in enumerate(results.pose_landmarks.landmark):
                if 10<id<17:
                    h, w, c = img.shape
                    x, y = int(lm.x*w) ,int(lm.y*h)
                    cv2.circle(img_og, (x,y), 3 , (0,255,0), cv2.FILLED)
                    landmarks.update({str(id)+"_x":lm.x})
                    landmarks.update({str(id)+"_y":lm.y})
                    landmarks.update({str(id)+"_z":lm.z})

                    
        cv2.imshow("actor", img_og)
        cv2.imshow("immitator", img2)
        cv2.waitKey(1)


        # Update arm postion of actor
        current_time=time.time()
        if current_time - previous_time >10:
            for joint_parameter in joint_parameters:
                random_position = random()*(joint_parameter[2]-joint_parameter[1])+joint_parameter[1]
                pepper2.setAngles(joint_parameter[0],random_position,1)
            previous_time=current_time 
            changed=False

        
        if current_time - previous_time >5 and not changed:
            if len(landmarks) == 18:
                changed = True
                x_data = [[landmarks[key] for key in landmarks]]
                y_data=model.predict(x_data)
                for i,joint_parameter in enumerate(joint_parameters):
                    pepper.setAngles(joint_parameter[0],clamp(y_data[0,i],joint_parameter[1],joint_parameter[2]),1)


except KeyboardInterrupt:
    pass
finally:
    cv2.destroyAllWindows()

In [ ]:
# sklearn multioutputregressor model

landmarks = dict()
previous_time = time.time()
model = pkl.load(open("model.pkl", 'rb'))
changed = False
try:
    while True:
        # Get pose from camera
        img = pepper.getCameraFrame(handle2)
        img2 = pepper2.getCameraFrame(handle3)
        img_og = img.copy()
        results = pd.getPose(img,draw_joints =True, draw_connections = True, only_arms_connections = True)
        if results.pose_landmarks:
            for id,lm in enumerate(results.pose_landmarks.landmark):
                if 10<id<17:
                    h, w, c = img.shape
                    x, y = int(lm.x*w) ,int(lm.y*h)
                    cv2.circle(img_og, (x,y), 3 , (0,255,0), cv2.FILLED)
                    landmarks.update({str(id)+"_x":lm.x})
                    landmarks.update({str(id)+"_y":lm.y})
                    landmarks.update({str(id)+"_z":lm.z})

        cv2.imshow("actor", img_og)
        cv2.imshow("immitator", img2)
        cv2.waitKey(1)


        # Update arm postion of actor
        current_time=time.time()
        if current_time - previous_time >10:
            for joint_parameter in joint_parameters:
                random_position = random()*(joint_parameter[2]-joint_parameter[1])+joint_parameter[1]
                pepper2.setAngles(joint_parameter[0],random_position,1)
            previous_time=current_time 
            changed = False

        
        if current_time - previous_time >5 and not changed:
            if len(landmarks) == 18:
                print("changing")
                changed = True
                x_data = [[landmarks[key] for key in landmarks]]
                y_data = model.predict(x_data)
                for i,joint_parameter in enumerate(joint_parameters):
                    pepper.setAngles(joint_parameter[0],clamp(y_data[0,i],joint_parameter[1],joint_parameter[2]),1)

except KeyboardInterrupt:
    pass
finally:
    cv2.destroyAllWindows()

In [ ]:
simulation_manager.stopSimulation(client_id)